In [ ]:
import os
import sqlite3

import openai
from polars import read_parquet
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest

openai.api_key = os.getenv("OPENAI_API_KEY")

df = read_parquet("../data/interim/srag_2019_2024.parquet")
DATA_COLS = {col: str(df[col].dtype) for col in df.columns}
df.head()

In [ ]:
class AIQueryAgent:
    """
    Um agente que recebe perguntas em linguagem natural,
    traduz para SQL, consulta um SQLite, sumariza os resultados
    e busca notícias relacionadas no Qdrant.
    """

    def __init__(
        self,
        sqlite_path: str,
        qdrant_url: str,
        qdrant_collection: str,
        embedding_model: str = "text-embedding-ada-002",
        sql_model: str = "gpt-3.5-turbo",
        summarization_model: str = "gpt-3.5-turbo",
        final_model: str = "gpt-3.5-turbo",
    ) -> None:
        """
        :param sqlite_path: caminho para o arquivo .db do SQLite
        :param qdrant_url: URL do servidor Qdrant
        :param qdrant_collection: nome da coleção onde estão as notícias
        :param embedding_model: modelo de embedding da OpenAI
        :param sql_model: modelo de LLM para tradução NL→SQL
        :param summarization_model: modelo de LLM para resumo de resultados
        :param final_model: modelo de LLM para montar a resposta final
        """
        self.sqlite_path = sqlite_path
        self.qdrant = QdrantClient(url=qdrant_url)
        self.qdrant_collection = qdrant_collection
        self.embedding_model = embedding_model
        self.sql_model = sql_model
        self.summarization_model = summarization_model
        self.final_model = final_model

    def _execute_sql(self, sql: str) -> list[tuple]:
        """Executa a query SQL no banco SQLite e retorna todas as linhas."""
        conn = sqlite3.connect(self.sqlite_path)
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        conn.close()
        return rows

    def _llm_chat(self, model: str, messages: list[dict]) -> str:
        """Chama o endpoint chat completions da OpenAI."""
        return (
            openai.ChatCompletion.create(model=model, messages=messages)
            .choices[0]
            .message.content
        )

    def _natural_to_sql(self, question: str) -> str:
        """
        Traduz a pergunta em linguagem natural para uma consulta SQL válida.
        """
        return self._llm_chat(
            self.sql_model,
            [
                {
                    "role": "system",
                    "content": "Você é um gerador de consultas SQL para SQLite.",
                },
                {
                    "role": "user",
                    "content": f"Transforme em SQL para SQLite:\n\n{question}",
                },
            ],
        ).strip()

    def _summarize(self, question: str, results: list[tuple]) -> str:
        """
        Gera um pequeno resumo explicativo dos resultados obtidos.
        """
        return self._llm_chat(
            self.summarization_model,
            [
                {
                    "role": "system",
                    "content": "Você é um assistente que resume resultados de tabelas.",
                },
                {
                    "role": "user",
                    "content": (
                        "Pergunta:\n"
                        + question
                        + "\n\nResultados (como lista de tuplas):\n"
                        + repr(results)
                        + "\n\nForneça um parágrafo curto explicando o que esses dados mostram."
                    ),
                },
            ],
        ).strip()

    def _get_embedding(self, text: str) -> list[float]:
        """Gera embedding para um texto usando a API da OpenAI."""
        return (
            openai.Embedding.create(model=self.embedding_model, input=[text])
            .data[0]
            .embedding
        )

    def _search_news(self, embedding: list[float], limit: int = 5) -> list[dict]:
        """
        Busca notícias relacionadas no Qdrant, usando cosine similarity.
        Retorna lista de hits (id, payload).
        """
        return [
            {"id": hit.id, "score": hit.score, "payload": hit.payload}
            for hit in self.qdrant.search(
                collection_name=self.qdrant_collection,
                query_vector=embedding,
                limit=limit,
                with_payload=True,
            )
        ]

    def ask(self, question: str) -> str:
        """
        Ciclo principal: recebe pergunta, executa SQL,
        resume, busca notícias e devolve resposta final.
        """
        sql = self._natural_to_sql(question)
        summary = self._summarize(question, self._execute_sql(sql))
        return self._llm_chat(
            self.final_model,
            [
                {
                    "role": "system",
                    "content": "Você é um assistente que responde perguntas com base em dados tabulares e notícias relacionadas.",
                },
                {
                    "role": "user",
                    "content": f"Pergunta: {question}\n\n"
                    f"Consulta SQL gerada:\n{sql}\n\n"
                    f"Resumo dos dados:\n{summary}\n\n"
                    f"Notícias relacionadas (id e título):\n"
                    + "\n".join(
                        f"- {hit['id']}: {hit['payload'].get('title', '(sem título)')}"
                        for hit in self._search_news(self._get_embedding(summary))
                    ),
                },
            ],
        )

In [ ]:
agent = AIQueryAgent(
    sqlite_path="../data/interim/srag_2019_2024.db",
    qdrant_url="http://localhost:6333",
    qdrant_api_key=None,
    qdrant_collection="noticias",
)
agent.ask("what was the month with more cases of sars in 2019")